# Deterministic Model

In [42]:
# -*- coding: utf-8 -*-
"""first model final optimization results.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1oGIQrCUbaymuIryqcEBkPMgWJ7YWS2O3
"""

import pulp as lp
import numpy as np
import pickle
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 100  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 0  # Weight for PGC
beta = 1.0  # Weight for TGC

Mean_Data = [ 44,  65,  97,  76,  42, 94, 160, 171, 77, 91, 75, 161]

# Randomly generated data
# waste_generated = {d: {day: Mean_Data[d] for day in days} for d in districts}
waste_generated = {d: {day: random.uniform(5, 20) for day in days} for d in districts}
frequency_required = {d: 3 for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot on day day
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                               ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
for d in districts:
    for day in days:
        model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) - 1, f"AbsPos_{d}_{day}"
        model_fixed += abs_neg[(d, day)] >= -(g_fixed[(d, day)] * (1 / waste_generated[d][day]) - 1), f"AbsNeg_{d}_{day}"
        model_fixed += abs_diff[(d, day)] == abs_pos[(d, day)] + abs_neg[(d, day)], f"AbsDiff_{d}_{day}"


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * waste_generated[d][day] /
                    truck_capacity for d in districts for t in range(total_trucks) for k in timeslots for day in days)

def PGC():
    return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)

# Define objective
model_fixed += alpha * TCT(x_fixed) + gamma * PGC() - beta * TGC(g_fixed)

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]
                                                     for t in range(total_trucks) for k in timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
    model_fixed += (2 <=lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) <= 3) , f"TotalPickupFrequency_Fixed_{d}"

# Equity constraint as a soft constraint with tolerance
#for d in districts:
  #   model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * SEI[d] for t in range(total_trucks) for k in timeslots for day in days) <= 1.5 * frequency_required[d], f"EquityConstraint_{d}"

# Truck capacity and availability constraints
for t in range(total_trucks):
    for d in districts:
        for k in timeslots:
            for day in days:
                model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
for t in range(total_trucks):
    model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0,1,2,3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve()

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics

final_solution = np.zeros((12,total_trucks,2,7))
totalused = 0
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in range(2):
                if lp.value(x_fixed[(d, t, timeslots[k], day)]) > 0 :
                    totalused += 1
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, timeslots[k], day)])}")
                    final_solution[d][t][k][day] += lp.value(x_fixed[(d,t,timeslots[k],day)])

with open("Deterministic_Solution.pkl", "wb") as file:
    pickle.dump(final_solution, file)

print(totalused)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/5abf3462115e44b89c8476bdb7bccab9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/5abf3462115e44b89c8476bdb7bccab9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17253 COLUMNS
At line 122410 RHS
At line 139659 BOUNDS
At line 157660 ENDATA
Problem MODEL has 17248 rows, 18336 columns and 52272 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -579.423 - 0.03 seconds
Cgl0004I processed model has 3 rows, 2159 columns (2159 integer (2130 of which binary)) and 2159 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -313.895
Cbc0038I Cleaned solution of -313.895
Cbc0038I Be

In [14]:
# converter

import pickle
import numpy as np

with open("Prioritize TCT TGC PGC truck sharing.pkl", "rb") as file:
    loaded_data = pickle.load(file)

with open("means.pkl", "rb") as file:
    Mean = pickle.load(file)

print(Mean)
schedule = np.zeros((12, 7, 2))

for day in range(7):
    for district_dum in range(12):
        for slot in range(2):
            for car in range(loaded_data.shape[1]):
                schedule[district_dum][day][slot] += loaded_data[district_dum][car][slot][day]


with open("Prioritize TCT TGC PGC truck sharing_Converted.pkl", "wb") as file:
    pickle.dump(schedule, file)

print(schedule)

[ 43.57303419  65.53205128  96.98660256  76.66292735  42.92226496
  94.48630342 160.81747863 171.82707265  77.58852564  91.11824786
  75.12809829 161.76136752]
[[[0. 0.]
  [1. 0.]
  [0. 0.]
  [0. 1.]
  [0. 0.]
  [0. 0.]
  [0. 1.]]

 [[0. 0.]
  [1. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 2.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [3. 0.]
  [0. 0.]
  [2. 1.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [2. 0.]
  [1. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 2.]]

 [[1. 1.]
  [1. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 1.]
  [0. 0.]
  [1. 0.]
  [4. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 2.]
  [0. 0.]
  [0. 0.]
  [0. 2.]
  [4. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 1.]
  [0. 0.]
  [7. 0.]
  [0. 0.]
  [0. 0.]
  [0. 3.]
  [0. 0.]]

 [[1. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [2. 0.]
  [2. 0.]]

 [[0. 0.]
  [0. 0.]
  [2. 0.]
  [2. 2.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[1. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 4.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [5. 4.]
  [0. 0.

In [12]:
import pulp as lp
import random
import pickle
import numpy as np

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 1.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]
random.seed(42)
# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_shared = lp.LpProblem("Truck_Shared_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_shared = lp.LpVariable.dicts("x_shared", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                            for k in timeslots for day in days), 0, 1, lp.LpBinary)
# y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
#                                   for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for time slot k on day day
# _shared_new = lp.LpVariable.dicts("y_shared_new",
#                                  ((d, t, )  for d in districts for t in range(total_trucks)) , 0, 1, lp.LpBinary)
g_shared = lp.LpVariable.dicts("g_shared", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_shared[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_shared += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)
    # Constraint to express absolute deviation from mean
    model_shared += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_shared += alpha * TCT(x_shared) - beta * TGC(g_shared) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_shared += g_shared[(d, day)] == lp.lpSum(
            x_shared[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_shared_{d}_{day}"
    # Frequency requirement constraints
# model_shared+= lp.lpSum(
#      x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_shared+= lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                        days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            model_shared += schedule_used[(d, day, k)] <= lp.lpSum(
                x_shared[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_shared += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_shared[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#   days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)(loose)
# for t in range(total_trucks):
#   model_shared += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"
# for t in range(total_trucks):
#   for day in days:
#     for k in timeslots:
#        model_shared += lp.lpSum(x_shared[()])
# Each truck must be assigned to exactly one district on each time slot on each distinctive day of the week


# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_shared += lp.lpSum(x_shared[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
                                                                                                  3], f"Truck_Frequency_{d}_{t}"
# Each truck can only take one certain time slot on a specific day in one district, not the other districts
for t in range(total_trucks):
    for day in days:
        for k in timeslots:
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for d in districts) <= 1

# Solve the model

model_shared.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Truck Sharing Model:")
print("Status:", lp.LpStatus[model_shared.status])
print("Objective Value:", lp.value(model_shared.objective))

with open("rioritize TCT TGC PGC truck sharing_original.pkl", "wb") as file2:
    pickle.dump(lp.value, file2)

# Output assignment results for diagnostics
final_solution = np.zeros((12,total_trucks,2,7))
totalused = 0
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in range(2):
                if lp.value(x_shared[(d, t, timeslots[k], day)]) > 0 :
                    totalused += 1
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_shared[(d, t, timeslots[k], day)])}")
                    final_solution[d][t][k][day] += lp.value(x_shared[(d,t,timeslots[k],day)])

with open("Prioritize TCT TGC PGC truck sharing.pkl", "wb") as file:
    pickle.dump(final_solution, file)



print(totalused)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/4db48eb0f74943f0961f2ef5ac9de69d-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/4db48eb0f74943f0961f2ef5ac9de69d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7462 COLUMNS
At line 596868 RHS
At line 604326 BOUNDS
At line 688495 ENDATA
Problem MODEL has 7457 rows, 84277 columns and 336961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is 7.8218e-12 - 0.72 seconds
Cgl0008I 7000 inequality constraints converted to equality constraints
Cgl0003I 0 fixed, 64 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 7415 rows, 91247 colum

In [13]:
# Prioritize TGC truck sharing.pkl

import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 0.0  # Weight for TCT
gamma = 0.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]
random.seed(42)
# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_shared = lp.LpProblem("Truck_Shared_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_shared = lp.LpVariable.dicts("x_shared", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                            for k in timeslots for day in days), 0, 1, lp.LpBinary)
# y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
#                                   for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for time slot k on day day
# _shared_new = lp.LpVariable.dicts("y_shared_new",
#                                  ((d, t, )  for d in districts for t in range(total_trucks)) , 0, 1, lp.LpBinary)
g_shared = lp.LpVariable.dicts("g_shared", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_shared[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_shared += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)
    # Constraint to express absolute deviation from mean
    model_shared += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_shared += alpha * TCT(x_shared) - beta * TGC(g_shared) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_shared += g_shared[(d, day)] == lp.lpSum(
            x_shared[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_shared_{d}_{day}"
    # Frequency requirement constraints
# model_shared+= lp.lpSum(
#      x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_shared+= lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                        days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            model_shared += schedule_used[(d, day, k)] <= lp.lpSum(
                x_shared[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_shared += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_shared[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#   days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)(loose)
# for t in range(total_trucks):
#   model_shared += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"
# for t in range(total_trucks):
#   for day in days:
#     for k in timeslots:
#        model_shared += lp.lpSum(x_shared[()])
# Each truck must be assigned to exactly one district on each time slot on each distinctive day of the week


# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_shared += lp.lpSum(x_shared[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
                                                                                                  3], f"Truck_Frequency_{d}_{t}"
# Each truck can only take one certain time slot on a specific day in one district, not the other districts
for t in range(total_trucks):
    for day in days:
        for k in timeslots:
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for d in districts) <= 1

# Solve the model

model_shared.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))



# Display results
print("Truck Sharing Model:")
print("Status:", lp.LpStatus[model_shared.status])
print("Objective Value:", lp.value(model_shared.objective))

with open("Prioritize TGC truck sharing.pkl", "wb") as file2:
    pickle.dump(lp.value, file2)

final_solution = np.zeros((12,total_trucks,2,7))
totalused = 0
# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_shared[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_shared[(d, t, k, day)])}")
                    totalused += 1
                    final_solution[d][t][k][day] += lp.value(x_shared[(d,t,timeslots[k],day)])


with open("Prioritize TGC truck sharing.pkl", "wb") as file:
    pickle.dump(final_solution, file)



# final_solution = np.zeros((12,total_trucks,2,7))
# totalused = 0

# totalused += 1
# final_solution[d][t][k][day] += lp.value(x_shared[(d,t,timeslots[k],day)])

# with open("Prioritize TGC truck sharing.pkl", "wb") as file:
#     pickle.dump(final_solution, file)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/1d5a279a01524529890fa6ec096ccca5-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/1d5a279a01524529890fa6ec096ccca5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7462 COLUMNS
At line 512844 RHS
At line 520302 BOUNDS
At line 604471 ENDATA
Problem MODEL has 7457 rows, 84277 columns and 336961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is -84000 - 2.94 seconds
Cgl0008I 7000 inequality constraints converted to equality constraints
Cgl0004I processed model has 7348 rows, 91168 columns (91168 integer (91168 of which binary)) and 259504 elements
Cutoff increment 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [15]:
# Prioritize TGC truck sharing

import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 0.0  # Weight for TCT
gamma = 0.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]
random.seed(42)
# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_shared = lp.LpProblem("Truck_Shared_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_shared = lp.LpVariable.dicts("x_shared", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                            for k in timeslots for day in days), 0, 1, lp.LpBinary)
# y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
#                                   for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for time slot k on day day
# _shared_new = lp.LpVariable.dicts("y_shared_new",
#                                  ((d, t, )  for d in districts for t in range(total_trucks)) , 0, 1, lp.LpBinary)
g_shared = lp.LpVariable.dicts("g_shared", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_shared[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_shared += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)
    # Constraint to express absolute deviation from mean
    model_shared += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_shared += alpha * TCT(x_shared) - beta * TGC(g_shared) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_shared += g_shared[(d, day)] == lp.lpSum(
            x_shared[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_shared_{d}_{day}"
    # Frequency requirement constraints
# model_shared+= lp.lpSum(
#      x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_shared+= lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                        days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            model_shared += schedule_used[(d, day, k)] <= lp.lpSum(
                x_shared[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_shared += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_shared[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#   days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)(loose)
# for t in range(total_trucks):
#   model_shared += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"
# for t in range(total_trucks):
#   for day in days:
#     for k in timeslots:
#        model_shared += lp.lpSum(x_shared[()])
# Each truck must be assigned to exactly one district on each time slot on each distinctive day of the week


# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_shared += lp.lpSum(x_shared[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
                                                                                                  3], f"Truck_Frequency_{d}_{t}"
# Each truck can only take one certain time slot on a specific day in one district, not the other districts
for t in range(total_trucks):
    for day in days:
        for k in timeslots:
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for d in districts) <= 1

# Solve the model

model_shared.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Truck Sharing Model:")
print("Status:", lp.LpStatus[model_shared.status])
print("Objective Value:", lp.value(model_shared.objective))


# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_shared[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_shared[(d, t, k, day)])}")




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/674ac3f5db5e4c21b7db2fd976d5ffd0-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/674ac3f5db5e4c21b7db2fd976d5ffd0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7462 COLUMNS
At line 512844 RHS
At line 520302 BOUNDS
At line 604471 ENDATA
Problem MODEL has 7457 rows, 84277 columns and 336961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is -84000 - 2.93 seconds
Cgl0008I 7000 inequality constraints converted to equality constraints
Cgl0004I processed model has 7348 rows, 91168 columns (91168 integer (91168 of which binary)) and 259504 elements
Cutoff increment 

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [17]:
# Prioritize TCT and TGC truck sharing


import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 0.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]
random.seed(42)
# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_shared = lp.LpProblem("Truck_Shared_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_shared = lp.LpVariable.dicts("x_shared", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                            for k in timeslots for day in days), 0, 1, lp.LpBinary)
# y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
#                                   for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for time slot k on day day
# _shared_new = lp.LpVariable.dicts("y_shared_new",
#                                  ((d, t, )  for d in districts for t in range(total_trucks)) , 0, 1, lp.LpBinary)
g_shared = lp.LpVariable.dicts("g_shared", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_shared[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_shared += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)
    # Constraint to express absolute deviation from mean
    model_shared += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_shared += alpha * TCT(x_shared) - beta * TGC(g_shared) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_shared += g_shared[(d, day)] == lp.lpSum(
            x_shared[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_shared_{d}_{day}"
    # Frequency requirement constraints
# model_shared+= lp.lpSum(
#      x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_shared+= lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                        days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            model_shared += schedule_used[(d, day, k)] <= lp.lpSum(
                x_shared[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_shared += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_shared += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_shared[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#   days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)(loose)
# for t in range(total_trucks):
#   model_shared += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"
# for t in range(total_trucks):
#   for day in days:
#     for k in timeslots:
#        model_shared += lp.lpSum(x_shared[()])
# Each truck must be assigned to exactly one district on each time slot on each distinctive day of the week


# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
for d in districts:
    for t in range(total_trucks):
        model_shared += lp.lpSum(x_shared[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
                                                                                                  3], f"Truck_Frequency_{d}_{t}"
# Each truck can only take one certain time slot on a specific day in one district, not the other districts
for t in range(total_trucks):
    for day in days:
        for k in timeslots:
            model_shared += lp.lpSum(x_shared[(d, t, k, day)] for d in districts) <= 1

# Solve the model

model_shared.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Truck Sharing Model:")
print("Status:", lp.LpStatus[model_shared.status])
print("Objective Value:", lp.value(model_shared.objective))

final_solution = np.zeros((12,total_trucks,2,7))
totalused = 0

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_shared[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_shared[(d, t, k, day)])}")
                    totalused += 1




with open("Prioritize TCT and TGC truck sharing.pkl", "wb") as file:
    pickle.dump(final_solution, file)





Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/4603a0693e1e4bc08c3c6b1d301802f6-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/4603a0693e1e4bc08c3c6b1d301802f6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7462 COLUMNS
At line 596844 RHS
At line 604302 BOUNDS
At line 688471 ENDATA
Problem MODEL has 7457 rows, 84277 columns and 336961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is -7.11038e-12 - 0.69 seconds
Cgl0008I 7000 inequality constraints converted to equality constraints
Cgl0004I processed model has 7348 rows, 91168 columns (91168 integer (91168 of which binary)) and 259504 elements
Cbc0045I No

In [18]:
#Prioritize TCT TGC PGC fixed model

import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 1.0  # Weight for PGC
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]

# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot in a week
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                                   ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_fixed[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_fixed += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)

    # Constraint to express absolute deviation from mean
    model_fixed += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_fixed += alpha * TCT(x_fixed) - beta * TGC(g_fixed) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(
            x_fixed[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
# model_fixed += lp.lpSum(
#     x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                       days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            # Upper bound constraint
            model_fixed += schedule_used[(d, day, k)] <= lp.lpSum(
                x_fixed[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_Upper_{d}_{day}_{k}"

            # Lower bound constraint
            model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleUsed_Lower_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
        # model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[(d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_fixed += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_fixed[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#  days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
# for t in range(total_trucks):
#  model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# Additional binary variable to indicate if a truck is assigned to a district at all
z = lp.LpVariable.dicts("z", ((t, d) for t in range(total_trucks) for d in districts), 0, 1, lp.LpBinary)

# Link x_fixed and z variables
for t in range(total_trucks):
    for d in districts:
        # Link x_fixed and z: if z[t, d] is 1, truck t can be assigned to district d on specific (day, timeslot) combinations
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for day in days for k in timeslots) <= z[(t, d)] * len(
            days) * len(timeslots)

# Ensure each truck is assigned to exactly one district across all days and timeslots
for t in range(total_trucks):
    model_fixed += lp.lpSum(z[(t, d)] for d in districts) == 1, f"OneDistrictP#_{t}"

    #   model_fixed += (lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) > 0)
# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
# for d in districts:
#  for t in range(total_trucks):
#      model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
#          3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_fixed[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, k, day)])}")




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/d1ee0b9fc6774c6cb7592d00dae03eca-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/d1ee0b9fc6774c6cb7592d00dae03eca-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6962 COLUMNS
At line 620368 RHS
At line 627326 BOUNDS
At line 717495 ENDATA
Problem MODEL has 6957 rows, 90277 columns and 348961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.64846e-12 - 0.28 seconds
Cgl0003I 0 fixed, 64 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 6915 rows, 90247 columns (90168 integer (90168 of which binary)) and 378703 elements
Cbc0038

In [19]:
# Prioritize TCT and TGC fixed model

import pulp as lp
import random
import math

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 1.0  # Weight for TCT
gamma = 0.0 # Weight for variance
beta = 1.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]

# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot in a week
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                                   ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_fixed[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_fixed += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)

    # Constraint to express absolute deviation from mean
    model_fixed += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_fixed += alpha * TCT(x_fixed) - beta * TGC(g_fixed) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(
            x_fixed[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
# model_fixed += lp.lpSum(
#     x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                       days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            # Upper bound constraint
            model_fixed += schedule_used[(d, day, k)] <= lp.lpSum(
                x_fixed[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_Upper_{d}_{day}_{k}"

            # Lower bound constraint
            model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleUsed_Lower_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
        # model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[(d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_fixed += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_fixed[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#  days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
# for t in range(total_trucks):
#  model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# Additional binary variable to indicate if a truck is assigned to a district at all
z = lp.LpVariable.dicts("z", ((t, d) for t in range(total_trucks) for d in districts), 0, 1, lp.LpBinary)

# Link x_fixed and z variables
for t in range(total_trucks):
    for d in districts:
        # Link x_fixed and z: if z[t, d] is 1, truck t can be assigned to district d on specific (day, timeslot) combinations
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for day in days for k in timeslots) <= z[(t, d)] * len(
            days) * len(timeslots)

# Ensure each truck is assigned to exactly one district across all days and timeslots
for t in range(total_trucks):
    model_fixed += lp.lpSum(z[(t, d)] for d in districts) == 1, f"OneDistrictP#_{t}"

    #   model_fixed += (lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) > 0)
# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
# for d in districts:
#  for t in range(total_trucks):
#      model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
#          3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_fixed[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, k, day)])}")




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/82414682a4544566b8032abfb3bb825f-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/82414682a4544566b8032abfb3bb825f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6962 COLUMNS
At line 620344 RHS
At line 627302 BOUNDS
At line 717471 ENDATA
Problem MODEL has 6957 rows, 90277 columns and 348961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.97176e-13 - 0.28 seconds
Cgl0004I processed model has 6848 rows, 90168 columns (90168 integer (90168 of which binary)) and 264504 elements
Cbc0045I No integer variables out of 90168 objects (90168 integer) have costs
Cbc00

In [20]:
#Prioritize PGC fixed model

import pulp as lp
import random

# Parameters
districts = range(12)  # 12 districts
days = range(7)  # Days in a week
timeslots = ["Morning", "Evening"]  # Two timeslots
total_trucks = 500  # Adjusted for feasibility
truck_capacity = 12  # Each truck’s capacity in tons
truck_collection_time_per_ton = 1  # Collection time per ton (for simplicity)
alpha = 0.0  # Weight for TCT
gamma = 1.0  # Weight for PGC
beta = 0.0  # Weight for TGC

lower_limit = [27, 32, 56, 53, 29, 62, 79, 85, 39, 57, 51, 99]
upper_limit = [60, 99, 138, 100, 57, 127, 243, 259, 116, 125, 99, 224]

# Randomly generated data
waste_generated = {d: {day: random.uniform(lower_limit[d], upper_limit[d]) for day in days} for d in districts}
# {d: {day: random.uniform(5, 25) for day in days} for d in districts}
frequency_required = {d: random.choice([2, 3]) for d in districts}
SEI = {d: random.uniform(0, 1) for d in districts}

# Initialize Model
model_fixed = lp.LpProblem("Fixed_Assignment_Model", lp.LpMinimize)

# Decision variables: binary and non-negative
x_fixed = lp.LpVariable.dicts("x_fixed", ((d, t, k, day) for d in districts for t in range(total_trucks)
                                          for k in timeslots for day in days), 0, 1, lp.LpBinary)
y_shared = lp.LpVariable.dicts("y_shared", ((d1, d2, t, day) for d1 in districts for d2 in districts if d1 != d2
                                            for t in range(total_trucks) for day in days), 0, 1, lp.LpBinary)
# Decision variable: y_shared_new = 1 if truck t is assigned to district d for any timeslot in a week
y_shared_new = lp.LpVariable.dicts("y_shared_new",
                                   ((d, t) for d in districts for t in range(total_trucks)), 0, 1, lp.LpBinary)
g_fixed = lp.LpVariable.dicts("g_fixed", ((d, day) for d in districts for day in days), 0)  # Non-negative variable

# Auxiliary variable for absolute value in PGC calculation
# Introduce two new variables for the absolute difference
# abs_pos = lp.LpVariable.dicts("abs_pos", ((d, day) for d in districts for day in days), 0)
# abs_neg = lp.LpVariable.dicts("abs_neg", ((d, day) for d in districts for day in days), 0)
# abs_diff = lp.LpVariable.dicts("abs_diff", ((d, day) for d in districts for day in days), 0)

# Absolute difference formulation
# for d in districts:
#   for day in days:
#      model_fixed += abs_pos[(d, day)] >= g_fixed[(d, day)] * (1 / waste_generated[d][day]) == 1

# Variance function using linear approximation (absolute deviations from mean)

proportions = []

# Loop over each district to compute the collected/generated ratio
for d in districts:
    # Total waste generated by district `d`
    total_waste_generated_d = sum(waste_generated[d][day] for day in days)

    # Total garbage collected for district `d` across all days
    total_garbage_collected_d = lp.lpSum(g_fixed[(d, day)] for day in days)

    # Calculate ratio if total waste generated is non-zero
    if total_waste_generated_d != 0:
        ratio = total_garbage_collected_d / total_waste_generated_d
        proportions.append(ratio)

# Linear approximation using absolute deviations from mean
# Auxiliary variable to represent the mean of proportions
mean_proportion = lp.LpVariable("mean_proportion", lowBound=0)

# Add constraint for mean calculation
model_fixed += (mean_proportion == lp.lpSum(proportions) / len(proportions))

# List to hold absolute deviations
abs_deviations = []

for i, proportion in enumerate(proportions):
    # Auxiliary variables for positive and negative deviations
    pos_dev = lp.LpVariable(f"pos_dev_{i}", lowBound=0)
    neg_dev = lp.LpVariable(f"neg_dev_{i}", lowBound=0)

    # Constraint to express absolute deviation from mean
    model_fixed += (proportion - mean_proportion == pos_dev - neg_dev, f"Deviation_{i}")

    # Add positive and negative deviations to get the absolute deviation
    abs_deviation = pos_dev + neg_dev
    abs_deviations.append(abs_deviation)


# Objective functions
def TCT(x):
    return lp.lpSum(x[(d, t, k, day)] * truck_collection_time_per_ton * truck_capacity for d in districts for t in
                    range(total_trucks) for k in timeslots for day in days)


# def PGC():
#   return lp.lpSum(abs_diff[(d, day)] for d in districts for day in days)

def TGC(g):
    return lp.lpSum(g[(d, day)] for d in districts for day in days)


# Define an objective function that includes minimizing absolute deviations
model_fixed += alpha * TCT(x_fixed) - beta * TGC(g_fixed) + gamma * (lp.lpSum(abs_deviations) / len(proportions))

# Constraints
for d in districts:
    for day in days:
        # Garbage collection constraints
        model_fixed += g_fixed[(d, day)] == lp.lpSum(
            x_fixed[(d, t, k, day)] * truck_capacity for t in range(total_trucks) for k in
            timeslots), f"GarbageCollection_Fixed_{d}_{day}"
    # Frequency requirement constraints
# model_fixed += lp.lpSum(
#     x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in days) >= 2
# model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks) for k in timeslots for day in
#                       days) <= 3, f"TotalPickupFrequency_Fixed_{d}"

# Step 1: Create a binary variable to track if a schedule (day, timeslot) is used in a district
schedule_used = lp.LpVariable.dicts("schedule_used",
                                    ((d, day, k) for d in districts for day in days for k in timeslots),
                                    0, 1, lp.LpBinary)

# Step 2: Set `schedule_used` to 1 if any truck is assigned to a district on a given (day, timeslot)
for d in districts:
    for day in days:
        for k in timeslots:
            # Upper bound constraint
            model_fixed += schedule_used[(d, day, k)] <= lp.lpSum(
                x_fixed[(d, t, k, day)] for t in range(total_trucks)), f"ScheduleUsed_Upper_{d}_{day}_{k}"

            # Lower bound constraint
            model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[
                (d, day, k)] * total_trucks, f"ScheduleUsed_Lower_{d}_{day}_{k}"

            # If any truck is assigned to (day, timeslot) for district `d`, then `schedule_used` must be 1
        # model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for t in range(total_trucks)) <= schedule_used[(d, day, k)] * total_trucks, f"ScheduleAssignment_{d}_{day}_{k}"

# Step 3: Limit the total number of distinct schedules to 2 or 3 for each district
for d in districts:
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) >= 2, f"MinSchedules_{d}"
    model_fixed += lp.lpSum(schedule_used[(d, day, k)] for day in days for k in timeslots) <= 3, f"MaxSchedules_{d}"

# Equity constraint as a soft constraint with tolerance
# for d in districts:
#    model_fixed += (frequency_required[d] / (sum(waste_generated[d]) * SEI[d])) == 0.01

# Truck capacity and availability constraints
# model_fixed += lp.lpSum(waste_generated[d][day] for d in districts for day in days) >= truck_capacity * lp.lpSum(
#   x_fixed[(d, t, k, day)] for d in districts for t in range(total_trucks) for k in timeslots for day in
#  days), "TotalTruckCapacity"

# for t in range(total_trucks):
#   for d in districts:
#       for k in timeslots:
#          for day in days:
#             model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] * waste_generated[d][day]) <= truck_capacity, f"TruckCapacity_{t}_{d}_{k}_{day}"

# 2. Each truck must be assigned to exactly one district for the whole week (across all days and timeslots)
# for t in range(total_trucks):
#  model_fixed += lp.lpSum(y_shared_new[(d, t)] for d in districts) == 1, f"OneDistrictPerTruck_{t}"

# Additional binary variable to indicate if a truck is assigned to a district at all
z = lp.LpVariable.dicts("z", ((t, d) for t in range(total_trucks) for d in districts), 0, 1, lp.LpBinary)

# Link x_fixed and z variables
for t in range(total_trucks):
    for d in districts:
        # Link x_fixed and z: if z[t, d] is 1, truck t can be assigned to district d on specific (day, timeslot) combinations
        model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for day in days for k in timeslots) <= z[(t, d)] * len(
            days) * len(timeslots)

# Ensure each truck is assigned to exactly one district across all days and timeslots
for t in range(total_trucks):
    model_fixed += lp.lpSum(z[(t, d)] for d in districts) == 1, f"OneDistrictP#_{t}"

    #   model_fixed += (lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) > 0)
# 3. Each district can have trucks operating 0, 1, 2, or 3 times during the week
# for d in districts:
#  for t in range(total_trucks):
#      model_fixed += lp.lpSum(x_fixed[(d, t, k, day)] for k in timeslots for day in days) in [0, 1, 2,
#          3], f"Truck_Frequency_{d}_{t}"

# Solve the model
model_fixed.solve(lp.PULP_CBC_CMD(msg=1, timeLimit=100))

# Display results
print("Fixed Assignment Model:")
print("Status:", lp.LpStatus[model_fixed.status])
print("Objective Value:", lp.value(model_fixed.objective))

# Output assignment results for diagnostics
for d in districts:
    for day in days:
        for t in range(total_trucks):
            for k in timeslots:
                if lp.value(x_fixed[(d, t, k, day)]) > 0:
                    print(f"District {d}, Truck {t}, {k}, Day {day}: Assignment = {lp.value(x_fixed[(d, t, k, day)])}")




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/homebrew/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/508a42f81cbb4c519c996d61dd691385-pulp.mps -sec 100 -timeMode elapsed -branch -printingOptions all -solution /var/folders/ql/dklml2ln7dnf6l874y2q745w0000gn/T/508a42f81cbb4c519c996d61dd691385-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6962 COLUMNS
At line 536284 RHS
At line 543242 BOUNDS
At line 633411 ENDATA
Problem MODEL has 6957 rows, 90277 columns and 348961 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 100
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.72 seconds
Cgl0003I 0 fixed, 64 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 6915 rows, 90247 columns (90168 integer (90168 of which binary)) and 378703 elements
Cbc0038I Initial 